# TP 3 (17Co2024)
#### Integrantes
- Adassus, Luciano
- de Pedro Mermier, Ignacio
- Cagua, Jonathan

# Enunciado
Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en Material_TPs/TP3/images a partir del template Material_TPs/TP3/template

# Parte 1: Obtener una detección del logo en cada imagen sin falsos positivos

Para realizar la detección de un logo dentro de una imagen, vamos a aplicar el mecanismo denominado "Template Matching", el cual se caracteriza por tener una imagen denominado "pattern" o "template" la cual se encuentra contenida, junto con otros elementos, dentro de la imagen objetivo. El objetivo es ir "barriendo" la imagen objetivo con el "template" e ir comparando la intersección por medio de una métrica. Luego de barrer la totalidad de la imagen objetivo, debemos obtener la locación que maximiza (o minimiza) dicha métrica, ya que esa será la zona donde indicaremos que ocurre la detección.

Por naturaleza, este procedimiento se caracteriza por requerir un procedimiento adicional para que sea invariante a escala y, para este caso en particualar, como tenemos distintas imagenes con distintos tamaños y factores de forma, será necesaria realizar un escalamiento inteligente del "template" para poder realizar la detección sin falsos positivos.

Otra caracteristica de este procedimiento es la necesidad de tener el "template" con la misma orientación en la imagen objetivo, es decir, no es invariante a la rotación. Sin embargo, todas las imágenes suminstradas poseen el logo de Coca-Coca (template) con la misma orientación (incluída el template), lo que nos permitirá realizar la detección de manera más sencilla.

Para generar la condición de invarianza a la escala, vamos a realizar el siguiente procedimiento:
1. Tomar el ancho total de la imagen objetivo como patrón y dividirlo "n" cantidad de veces
2. Iterar sobre cada sub-escala obtenida en el punto anterior
3. Escalar el template en función de cada sub-escala
4. Realizar el procedimiento de template matching propiamente dicho y obtener una metrica
5. De todas las detecciones obtenidas, nos vamos a quedar con aquella que maximice la métrica determinada (Coeficiente de correlación normalizado o Correlación cruzada normalizada)
6. Ubicar un bounding box en la posición obtenida en el punto anterior con el factor de forma de la sub-escala que obtuvo dicha métrica


In [1]:
%matplotlib qt
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Cargamos el logo de Coca-Cola (template) y las imagenes objetivo

In [2]:
# Cargo el template
template = cv.imread('template/pattern.png')

# Cargo imagenes objetivo
coca_logo_1 = cv.imread('images/coca_logo_1.png')
coca_logo_2 = cv.imread('images/coca_logo_2.png')
coca_cola_logo = cv.imread('images/COCA-COLA-LOGO.jpg')
logo_1 = cv.imread('images/logo_1.png')
coca_multi = cv.imread('images/coca_multi.png')
coca_retro_1 = cv.imread('images/coca_retro_1.png')
coca_retro_2 = cv.imread('images/coca_retro_2.png')
 

In [3]:
type(template)

numpy.ndarray

Analizando el tamaño de cada imagen, vemos que las mismas varían de tamaño total y factor de forma entre sí y con respecto al template.

In [4]:
# Analizo los tamaños de las imagenes
print('Tamaño del template\n')
print('template',template.shape)
print('---------------------------')
print('Tamaño de las imagenes objetivo:\n')
print('coca_logo_1',coca_logo_1.shape)
print('coca_logo_2',coca_logo_2.shape)
print('coca_cola_logo',coca_cola_logo.shape)
print('logo_1',logo_1.shape)
print('coca_multi',coca_multi.shape)
print('coca_retro_1',coca_retro_1.shape)
print('coca_retro_2',coca_retro_2.shape)

Tamaño del template

template (175, 400, 3)
---------------------------
Tamaño de las imagenes objetivo:

coca_logo_1 (500, 207, 3)
coca_logo_2 (363, 233, 3)
coca_cola_logo (1389, 1389, 3)
logo_1 (450, 687, 3)
coca_multi (598, 799, 3)
coca_retro_1 (493, 715, 3)
coca_retro_2 (429, 715, 3)


Determinamos una función capaz de realizar un template matching sin depender de la escala de la imagen objetivo ni del template. Con el objetivo de optimizar la detección se realiza un pre-procesamiento de ambas imagenes:
- Pasar la imagen a escala de grises
- Aplicar un desenfoque gaussiano para eliminar el ruido
- Aplicar un mecanismo de detección de bordes para independizarse del fondo de las imagenes


In [5]:
# Funcion para detectar template matching con multiples escalas

def multiScaleTemplateMatching(template,image,edge_detector="canny",matchTempMethod = cv.TM_CCOEFF_NORMED,n_scales = 12):
    """
    Realiza el prodecimiento de Template Matching entre "template" e "image".
    
    Parámetros:
    - template: numpy.ndarray, imagen template/pattern en formato BGR
    - image: numpy.ndarray, imgagen objetivo en donde realizar la detección en formato BGR
    - edge_detector: str, mecanismo para realizar la detección de bordes ("canny" o "laplacian")
    - matchTempMethod: int, parámetro que especifica el metodo de comparación dentro de la función matchTemplate (ver https://docs.opencv.org/4.x/df/dfb/group__imgproc__object.html#ga3a7850640f1fe1f58fe91a2d7583695d)
    - n_scales: int, cantidad de sub-escalas en la que se dividirá la imagen objetivo para realizar la detección
    
    Retorna:
    - matchImg: numpy.ndarray, imagen objetivo + bounding box donde se realizó la detección
    - matchValue: float, resultado máximo (o minimo) de la función cv.matchTemplate que indica la detección
    """
        

    # Verifico que el mecanismo de detección de bordes sea Canny o Laplacian
    if edge_detector != "canny" and edge_detector != "laplacian":
        print('El parametro edge detector debe ser "canny" o "laplacian"')
        return
    
    # Paso las imagenes a escala de grises
    imgGray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    tmpGray = cv.cvtColor(template,cv.COLOR_BGR2GRAY)

    # Aplico Gaussian Blur sobre la imagen objetivo para sacar ruido
    imgGray = cv.GaussianBlur(imgGray,(5,5),0)

    # Aplica el filtro de detección de bordes
    if edge_detector == "canny":
        imgGray = cv.Canny(imgGray, 100, 120, apertureSize = 3)
    elif edge_detector == "laplacian":
        imgGray = cv.Laplacian(imgGray, cv.CV_8U)
    
    # Reescalo el template n_scales veces partiendo desde el ancho de la imagen
    th,tw = tmpGray.shape
    ih,iw = imgGray.shape
    scales = np.linspace(0, iw/tw, n_scales+1)[1:]

    # Listas para guardar resultados
    matching_results = list()
    minmax_results = list()
    template_shapes = list()
    
    for scale in scales:

        # Escalo el template
        tmpGray_scaled = cv.resize(tmpGray, None, fx=scale, fy=scale)

        # Aplico el Gaussian Blur sobre el template para sacar ruido
        tmpGray_scaled = cv.GaussianBlur(tmpGray_scaled,(3,3),0)
        
        # Aplica el filtro de detección de bordes
        if edge_detector == "canny":
            tmpGray_scaled = cv.Canny(tmpGray_scaled, 150, 180, apertureSize = 3)
        elif edge_detector == "laplacian":
            tmpGray_scaled = cv.Laplacian(tmpGray_scaled, cv.CV_8U)
        
        # Obtengo las dimensiones del template escalado
        tsh,tsw = tmpGray_scaled.shape
        
        # Aplico el template matching 
        result = cv.matchTemplate(imgGray, tmpGray_scaled, matchTempMethod)
        
        # Normalizo el resultado 
        result = (result-np.mean(result))/np.std(result)
        
        # Guardo el resultado
        matching_results.append(result)

        # Aplico la función minMaxLoc para luego generar el bounding box 
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
        
        #Guardo minMaxLocResult
        minmax_results.append([min_val, max_val, min_loc, max_loc])

        # Guardo las dimensiones del template para el bounding box
        template_shapes.append([tsh,tsw])

    # Obtengo el mejor resultado de todos matching para todos los casos y obtengo su ubicacion

    # En caso de utilizar Coeficiente de correlación o correlación cruzada (normalizada o no) busco el máximo
    if matchTempMethod == cv.TM_CCOEFF_NORMED or matchTempMethod == cv.TM_CCORR_NORMED or matchTempMethod == cv.TM_CCORR or matchTempMethod == cv.TM_CCOEFF:
        
        # Busco el maximo de maximos
        max_column = [sublist[1] for sublist in minmax_results]
        max_value = max(max_column)
        max_index = max_column.index(max_value)

        match_value = max_value
        match_index = max_index
        
       
    # En caso de utilizar suma de diferencias al cuadrado busco el minimo
    elif matchTempMethod == cv.TM_SQDIFF_NORMED:
        
        # Busco el minimo de minimos
        min_column = [sublist[0] for sublist in minmax_results]
        min_value = max(min_column)
        min_index = min_column.index(min_value)

        match_value = min_value
        match_index = min_index
        

    else:

        print('matchTempMethod error')
        return
    
    # Creo la imagen de salida basad en la imagen objetivo + Bounding box + Score
    matchImg = image.copy()
    topLeft = minmax_results[match_index][3]
    bottomRight = (minmax_results[match_index][3][0]+template_shapes[match_index][1],minmax_results[match_index][3][1]+template_shapes[match_index][0])
 
    cv.rectangle(matchImg, topLeft, bottomRight, (255,0,0), 2)
    #cv.putText(matchImg,f'Score {str(int(confidence_value*100))}%', (topLeft[0],topLeft[1]-6), cv.QT_FONT_NORMAL, 0.45, (255,0,0))  
  
    return matchImg,match_value


Probamos la función previamente presentada con distintas imagenes con el detector de bordes "Laplaciano" y metodo de detección "Coeficiente de correlacion normalizado"

In [6]:
# Template Matching usando TM_CCOEFF_NORMED
filter_border = "laplacian" #"canny"
print('BORDER_FILTER:\t\t LAPLACIAN')
print('MATCH PARAM:\t\t TM_CCOEFF_NORMED')
resultImg,matchValue = multiScaleTemplateMatching(template, coca_logo_1, filter_border,cv.TM_CCOEFF_NORMED,10)
print('coca_logo_1 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_logo_2,filter_border,cv.TM_CCOEFF_NORMED, 1)
print('coca_logo_2 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_cola_logo, filter_border,cv.TM_CCOEFF_NORMED,10)
print('coca_cola_logo metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, logo_1, filter_border,cv.TM_CCOEFF_NORMED,10)
print('logo_1 metric:\t\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_retro_1, filter_border,cv.TM_CCOEFF_NORMED,10)
print('coca_retro_1 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_retro_2, filter_border,cv.TM_CCOEFF_NORMED,10)
print('coca_retro_2 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_multi, filter_border,cv.TM_CCOEFF_NORMED,39)
print('coca_multi metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()


BORDER_FILTER:		 LAPLACIAN
MATCH PARAM:		 TM_CCOEFF_NORMED
coca_logo_1 metric:	6.93506383895874
coca_logo_2 metric:	2.856226682662964
coca_cola_logo metric:	5.459420680999756
logo_1 metric:		5.523893356323242
coca_retro_1 metric:	5.298094272613525
coca_retro_2 metric:	8.241572380065918
coca_multi metric:	4.4918718338012695


Probamos las distintas imagenes con el detector de bordes "Canny" y metodo de detección "Coeficiente de correlacion normalizado"

In [7]:
# Template Matching usando CANNY + TM_CCOEFF_NORMED
filter_border = "canny" #"canny"
print('BORDER_FILTER:\t\t CANNY')
print('MATCH PARAM:\t\t TM_CCOEFF_NORMED')
resultImg,matchValue = multiScaleTemplateMatching(template, coca_logo_1, filter_border,cv.TM_CCOEFF_NORMED,10)
print('coca_logo_1 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_logo_2,filter_border,cv.TM_CCOEFF_NORMED, 1)
print('coca_logo_2 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_cola_logo, filter_border,cv.TM_CCOEFF_NORMED,10)
print('coca_cola_logo metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, logo_1, filter_border,cv.TM_CCOEFF_NORMED,10)
print('logo_1 metric:\t\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_retro_1, filter_border,cv.TM_CCOEFF_NORMED,10)
print('coca_retro_1 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_retro_2, filter_border,cv.TM_CCOEFF_NORMED,10)
print('coca_retro_2 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_multi, filter_border,cv.TM_CCOEFF_NORMED,39)
print('coca_multi metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()



BORDER_FILTER:		 CANNY
MATCH PARAM:		 TM_CCOEFF_NORMED
coca_logo_1 metric:	9.57135009765625
coca_logo_2 metric:	3.1164133548736572
coca_cola_logo metric:	7.664137363433838
logo_1 metric:		6.827859878540039
coca_retro_1 metric:	4.688180923461914
coca_retro_2 metric:	11.194732666015625
coca_multi metric:	6.596279144287109


Probamos las distintas imagenes con el detector de bordes "Laplaciano" y metodo de detección "Correlacion Cruzada normalizada"

In [8]:
# Template Matching usando LAPLACIAN + TM_CCORR_NORMED
filter_border = "laplacian"
print('BORDER_FILTER:\t\t LAPLACIAN')
print('MATCH PARAM:\t\t TM_CCORR_NORMED')
resultImg,matchValue = multiScaleTemplateMatching(template, coca_logo_1, filter_border,cv.TM_CCORR_NORMED,10)
print('coca_logo_1 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_logo_2,filter_border,cv.TM_CCORR_NORMED, 1)
print('coca_logo_2 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_cola_logo, filter_border,cv.TM_CCORR_NORMED,10)
print('coca_cola_logo metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, logo_1, filter_border,cv.TM_CCORR_NORMED,10)
print('logo_1 metric:\t\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_retro_1, filter_border,cv.TM_CCORR_NORMED,10)
print('coca_retro_1 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_retro_2, filter_border,cv.TM_CCORR_NORMED,10)
print('coca_retro_2 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_multi, filter_border,cv.TM_CCORR_NORMED,39)
print('coca_multi metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()


BORDER_FILTER:		 LAPLACIAN
MATCH PARAM:		 TM_CCORR_NORMED
coca_logo_1 metric:	5.737558364868164
coca_logo_2 metric:	2.7343356609344482
coca_cola_logo metric:	4.2394118309021
logo_1 metric:		4.945406436920166
coca_retro_1 metric:	5.6729655265808105
coca_retro_2 metric:	6.234921455383301
coca_multi metric:	4.145476341247559


Probamos las distintas imagenes con el detector de bordes "Canny" y metodo de detección "Correlacion Cruzada normalizada"

In [9]:
# Template Matching usando LAPLACIAN + TM_CCORR_NORMED
filter_border = "canny"
print('BORDER_FILTER:\t\t CANNY')
print('MATCH PARAM:\t\t TM_CCORR_NORMED')
resultImg,matchValue = multiScaleTemplateMatching(template, coca_logo_1, filter_border,cv.TM_CCORR_NORMED,10)
print('coca_logo_1 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_logo_2,filter_border,cv.TM_CCORR_NORMED, 1)
print('coca_logo_2 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_cola_logo, filter_border,cv.TM_CCORR_NORMED,10)
print('coca_cola_logo metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, logo_1, filter_border,cv.TM_CCORR_NORMED,10)
print('logo_1 metric:\t\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_retro_1, filter_border,cv.TM_CCORR_NORMED,12)
print('coca_retro_1 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_retro_2, filter_border,cv.TM_CCORR_NORMED,10)
print('coca_retro_2 metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg,matchValue = multiScaleTemplateMatching(template, coca_multi, filter_border,cv.TM_CCORR_NORMED,39)
print('coca_multi metric:\t'+str(matchValue))
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()


BORDER_FILTER:		 CANNY
MATCH PARAM:		 TM_CCORR_NORMED
coca_logo_1 metric:	6.639416217803955
coca_logo_2 metric:	2.915717124938965
coca_cola_logo metric:	6.31380033493042
logo_1 metric:		4.614475727081299
coca_retro_1 metric:	4.398003101348877
coca_retro_2 metric:	7.918277263641357
coca_multi metric:	4.641242027282715


## Analisis de resultados


| Imagen            | CCOEFF + LAPLACIAN | CCOEFF + CANNY | CCORR + LAPLACIAN | CCORR + CANNY |
|-------------------|--------------------|----------------|-------------------|---------------|
| coca_logo_1       | 6.93               | 9.57           | 5.73              | 6.63          |
| coca_logo_2       | 2.85               | 3.11           | 2.73              | 2.91          |
| coca_cola_logo    | 5.45               | 7.66           | 4.23              | 6.31          |
| logo_1            | 5.52               | 6.82           | 4.95              | 4.61          |
| coca_retro_1      | 5.29               | 4.68           | 5.67              | 3.04          |
| coca_retro_2      | 8.24               | 11.19          | 6.23              | 7.91          |
| coca_multi        | 4.49               | 6.59           | 4.14              | 4.64          |

Al anlizar los resultados de la tabla podemos observar que para la gran mayoría de los casos, la mejor combinación de métrica y filtro de detección de bordes es Coeficiente de Correlación normalizada y Canny, respectivamente.

Es necesario aclarar que para lograr el objetivo de evitar falsos positivos, fue necesario encontrar mediante prueba y error los valores de n_scales para cada imagen. En su mayoría, generar 10 sub-escalas fuen suficiente para realizar la detección correctamente. Sin embargo, para la imagen coca_logo_2, la cual presenta el logo levemente modificado en sus bordes izquierdo y derecho (por la curvatura de la lata), fue necesario utilizar un valor de "n_scales" de 1 (sin escalar) para obtener el resultado correcto.

Para el caso de la imagen "coca_multi", la cual presenta multiples logos en la imagen, se pudo obervar que al variar el valor de "n_scales", variaba la locación del máximo (y por ende bounding box) donde realizaba la detección.

### Conclusión 

Como conclusión podemos establecer que el mejor mecanismo para la detección de logos sin falsos positivos es la combinación de Correlación cruzada normalizada como métrica de detección y Canny como filtro de extracción de bordes. A su vez, se oberva que el mecanismo de "Template Matching" es muy suceptible a modificaciones del logo en la imagen objetivo, aunque estas sean pequeñas.

**Nota: Los valores de n_scales para los casos anteriores se encontraron de manera iterativa para cada uno de los casos**

# Parte 2: Plantear y validar un algoritmo para múltiples detecciones en la imagen "coca_multi.png" con el mismo témplate del ítem 1

Para este caso, vamos a uilizar como base la función implementada en el punto 1 y vamos a realizar una serie de modificaciones para permitir la detección múltiple. Las mismas son:
- Dentro del bucle de las sub-escalas del pattern vamos a guardar unicamente los resultados cuya métrica que se encuentran por encima (o debajo para el caso de suma de diferencias al cuadrado) de un umbral establecido por el usuario.
- Si la métrica es superior al umbral, vamos a determinar el nivel de confianza a partir del valor maximo establecido hasta ese momento
- Como voy a tener múltiples detecciones, en caso que exista solapamiento de bounding boxes, vamos a determinar el mejor de todos mediante la técnica "Non Max Suppression" (NMS) 

In [10]:
# Extension de la funcion multiScaleTemplateMatching para multiples detecciones
def multiScaleTemplateMatchingMultipleDetection(template,image,edge_detector="canny",matchTempMethod = cv.TM_CCOEFF_NORMED,n_scales = 12,threshold=3):
    """
    Realiza el prodecimiento de Template Matching múltiple entre "template" e "image".
    
    Parámetros:
    - template: numpy.ndarray, imagen template/pattern en formato BGR
    - image: numpy.ndarray, imgagen objetivo en donde realizar la detección en formato BGR
    - edge_detector: str, mecanismo para realizar la detección de bordes ("canny" o "laplacian")
    - matchTempMethod: int, parámetro que especifica el metodo de comparación dentro de la función matchTemplate (ver https://docs.opencv.org/4.x/df/dfb/group__imgproc__object.html#ga3a7850640f1fe1f58fe91a2d7583695d)
    - n_scales: int, cantidad de sub-escalas en la que se dividirá la imagen objetivo para realizar la detección
    - threshold: int o float, umbral que determina si la métrica obtenida corresponde con una detección o no.
    
    Retorna:
    - matchImg: numpy.ndarray, imagen objetivo + bounding box donde se realizó la detección
    """
    
    # Verifico que el mecanismo de detección de bordes sea Canny o Laplacian
    if edge_detector != "canny" and edge_detector != "laplacian":
        print('El parametro edge detector debe ser "canny" o "laplacian"')
        return
    
    # Paso las imagenes a escala de grises
    imgGray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    tmpGray = cv.cvtColor(template,cv.COLOR_BGR2GRAY)

    # Aplico Gaussian Blur sobre la imagen objetivo para sacar ruido
    imgGray = cv.GaussianBlur(imgGray,(5,5),0)
    
    # Aplica el filtro de detección de bordes
    if edge_detector == "canny":
        imgGray = cv.Canny(imgGray, 100, 120, apertureSize = 3)
    elif edge_detector == "laplacian":
        imgGray = cv.Laplacian(imgGray, cv.CV_8U)
    
    # Obtengo dimensiones de la imagen objetivo y del template para obtener las sub-escalas
    th,tw = tmpGray.shape
    ih,iw = imgGray.shape
    
    # Reescalo el template n_scales veces partiendo desde el ancho de la imagen
    scales = np.linspace(0, iw/tw, n_scales+1)[1:]

    matching_results = list()
    matching_locations = list()

    confidence_values = list()
        
    for scale in scales:

        # Escalo el template
        tmpGray_scaled = cv.resize(tmpGray, None, fx=scale, fy=scale)
        
        # Aplico Gaussian Blur sobre la imagen objetivo para sacar ruido
        tmpGray_scaled = cv.GaussianBlur(tmpGray_scaled,(3,3),0)
        
        # Aplica el filtro de detección de bordes
        if edge_detector == "canny":
            tmpGray_scaled = cv.Canny(tmpGray_scaled, 150, 180, apertureSize = 3)
        elif edge_detector == "laplacian":
            tmpGray_scaled = cv.Laplacian(tmpGray_scaled, cv.CV_8U)
        
        # Obtengo las dimensiones del template escalado para el bounding box
        tsh,tsw = tmpGray_scaled.shape

        # Aplico el template matching 
        result = cv.matchTemplate(imgGray, tmpGray_scaled, matchTempMethod)
        
        # Normalizo el resultado 
        result = (result-np.mean(result))/np.std(result)
        
        # Guardo el resultado
        matching_results.append(result)
        
        # Guardo los resultados por encima del threshold
        best_results_loc = np.where(result >= threshold)
        for location in zip(*best_results_loc[::-1]): 
            
            matching_locations.append([location[0], location[1], location[0]+tsw, location[1]+tsh])
            
            # Establezco el nivel de confianza de la deteccion
            confidence_value = result[location[::-1]]
            confidence_values.append(confidence_value/np.max(result))



    matchImg = image.copy()

    # Aplico NMS para eliminar solapamiento de bounding boxies con un threshold the 0.75
    indices = cv.dnn.NMSBoxes(bboxes=matching_locations, scores=confidence_values, score_threshold=0.75, nms_threshold=0.75)
    for i in indices:
        coord = matching_locations[i]
        
        cv.rectangle(matchImg, (coord[0],coord[1]), (coord[2], coord[3]), (255,0,0), 2)
        cv.putText(matchImg,f'Score {str(int(confidence_values[i]*100))}%', (coord[0],coord[1]-6), cv.QT_FONT_NORMAL, 0.45, (255,0,0))


    return matchImg

## Test 1
Probamos la detección del logo con detección de bordes por "Canny" y metodo de detección "Coeficiente de correlación"

### Resultado
18 detecciones sobre 19 totales (94.73%) y sin solapamientos

In [11]:
scale = 10
threshold = 4.6
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi,"canny",cv.TM_CCOEFF,scale,threshold)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
cv.destroyAllWindows()

## Test 2
Probamos la detección del logo con detección de bordes por "Laplaciano" y metodo de detección "Coeficiente de correlación"

### Resultado
12 detecciones sobre 19 totales (63.15%)

In [12]:
scale = 13
threshold = 5.4
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi,"laplacian",cv.TM_CCOEFF,scale,threshold)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
cv.destroyAllWindows()

## Test 3
Probamos la detección del logo con detección de bordes por "Canny" y metodo de detección "Correlación Cruzada"

### Resultado
18 detecciones sobre 19 totales (94.73%), con un único solapamiento

In [13]:
scale = 11
threshold = 3.6
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi,"canny",cv.TM_CCORR,scale,threshold)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
cv.destroyAllWindows()

## Test 4
Probamos la detección del logo con detección de bordes por "Laplaciano" y metodo de detección "Correlación Cruzada"

Resultado: 19 detecciones sobre 19 totales (100%), con 2 solapamientos 

In [14]:
scale = 13
threshold = 3.3
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi,"laplacian",cv.TM_CCORR,scale,threshold)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
cv.destroyAllWindows()

## Analisis de resultados


| Imagen            | CCOEFF + LAPLACIAN | CCOEFF + CANNY | CCORR + LAPLACIAN | CCORR + CANNY |
|-------------------|--------------------|----------------|-------------------|---------------|
| coca_multi        | 63.15 %            | 94.73 %        | 100 %             | 94.73 %       |


Al analizar los resultados, podemos establecer que la mejor combinación de métrica de detección y filtro de detección de bordes es Correlación Cruzada y Canny, respectivamente.

Es necesario aclarar que para lograr el objetivo de evitar falsos positivos, detectar la mayor cantidad logos correctemente y evitar solapamientos de bounding boxes, fue necesario encontrar mediante prueba y error los valores de n_scales y threshold para cada caso.
A pesar que los valores de escalas establecidos varían entre 10 y 13 y los valores de umbral entre 3.3 y 5.4, pequeños cambios en dichos valores generan resultados muy distintos. Finalmente es necesario aclarar que para este caso, la métrica de detección que mejoraba las detecciones fueron aquellas que no se encontraban normalizadas.

### Conclusión 

Como conclusión podemos establecer que el mejor mecanismo para la detección de logos sin falsos positivos para este caso en particular,es la combinación de Correlación cruzada como métrica de detección y Laplaciano como filtro de extracción de bordes.
Sin embargo, esta forma de detectar múltiples detecciones es muy suceptible a los valores de entrada, haciendo ineficiente su uso de manera generalizada. Por lo que se recomendaría ir por otro de tipo de algoritmos, como aquellos orientados a descriptores de imagenes.


# Parte 3: Generalizar el algoritmo del item 2 para todas las imágenes.

Para este último caso vamos a probar la función presentada en el punto 2 sobre las imagenes utilizadas en el punto 1, junto con dos imagenes extraídas de internet para testear levemente la misma.

## Test 1

Probamos la detección del logo con detección de bordes por "Canny" y metodo de detección "Correlación Cruzada" (multiple detección) y "Correlación Cruzada Normalizada" (detección singular)

In [15]:
# Template Matching usando TM_CCORR_NORMED
edge_detection = "canny" #"canny"
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_logo_1, edge_detection,cv.TM_CCORR_NORMED,10,5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_logo_2,edge_detection,cv.TM_CCORR_NORMED, 1,1.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_cola_logo, edge_detection,cv.TM_CCORR_NORMED,10,5.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, logo_1, edge_detection,cv.TM_CCORR_NORMED,10,4.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_retro_1, edge_detection,cv.TM_CCORR_NORMED,12,4.2)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_retro_2, edge_detection,cv.TM_CCORR_NORMED,10,5)
cv.imshow('resultImg',resultImg) 
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi, edge_detection,cv.TM_CCORR,11,3.6)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()

## Test 2

Probamos la detección del logo con detección de bordes por "Laplaciando" y metodo de detección "Correlación Cruzada" (multiple detección) y "Correlación Cruzada Normalizada" (detección singular)

In [16]:
edge_detection = "laplacian" #"canny"
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_logo_1, edge_detection,cv.TM_CCORR_NORMED,10,5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_logo_2,edge_detection,cv.TM_CCORR_NORMED, 1,1.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_cola_logo, edge_detection,cv.TM_CCORR_NORMED,10,4.2)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, logo_1, edge_detection,cv.TM_CCORR_NORMED,10,4.9)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_retro_1, edge_detection,cv.TM_CCORR_NORMED,12,4.2)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_retro_2, edge_detection,cv.TM_CCORR_NORMED,10,5)
cv.imshow('resultImg',resultImg) 
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi, edge_detection,cv.TM_CCORR,13,3.9)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()

## Test 3

Finalmente probamos la funcion "multiScaleTemplateMatchingMultipleDetection" con dos imagenes descargas de internet, utilizando como filtro de detección de bordes "Laplaciano" y "Correlación Cruzada" como métrica de detección.

In [17]:
coca_test = cv.imread('images/coca_test.png')
coca_test_2 = cv.imread('images/coca_test_2.png')
coca_test_3 = cv.imread('images/coca_test_3.png')

edge_detection = "laplacian"
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_test, edge_detection,cv.TM_CCORR,12,5.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_test_2, edge_detection,cv.TM_CCORR,4,3.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_test_3, edge_detection,cv.TM_CCORR,8,5.9)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
cv.destroyAllWindows()

## Conclusiones

Podemos observar a partir de los 3 tests planteados anteriormente que la función establecida en el punto anterior puede utilizarse tanto para detecciones múltiples como para detecciones singulares. Es necesario volver a aclarar que para todos los casos fue necesario establecer, mediante el mecanismo de prueba y error, los valores de cantidad de sub-escalas y umbral de detección para obtener los resultados deseados. A su vez, se observa en el Test No. 3, que la función generaliza bastante bien para otras imagenes pero como se mencionó en el punto 1, el mismo sigue siendo muy suceptible a peuqeños cambios del template dentro de la imagen objetivo (nuevamente no se detecta el logo en las latas por la curvatura de las mismas).